## End to end model:Classifier

In [ ]:
import h5py

features = np.array([10, 128, 128, 2048])
labels = np.array([10])

h5f = h5py.File('feature_label.h5', 'w')
h5f.create_dataset('features', data=features)
h5f.create_dataset('labels', data=labels)

h5f = h5py.File('data.h5','r')
b = h5f['dataset_1'][:]
h5f.close()
